In [1]:
import pandas as pd

In [2]:
class_labels = pd.read_csv("class_accuracy.csv")
pixel_count = pd.read_csv("label_pixel_counts.csv")

In [3]:
# Merge the pixel count and the class labels
merged = pd.merge(class_labels, pixel_count, left_on="GT", right_on="Label")

In [4]:
# Group by "4th_tier_ENG" and create lists of pixel counts from the "GT" column
grouped_gt_codes = merged.groupby("4th_tier_ENG").agg(
    GT_list=("GT", list),
    pixel_sum=("Pixel Count", sum)
).reset_index()



In [5]:
grouped_gt_codes# Which groups are above 100000 pixels?

grouped_gt_codes[grouped_gt_codes["pixel_sum"] > 50000] # Can you set all the pixel counts to 0 for the groups that are above 100000 pixels?

# Assuming your DataFrame is named `grouped_gt_codes`

# Filter rows where 'pixel_sum' is less than 50,000
filtered_gt = grouped_gt_codes[grouped_gt_codes['pixel_sum'] > 50000].reset_index(drop=True)
filtered_gt_list =  filtered_gt['GT_list'].tolist()

# Display the filtered GT_list
combined_gt_list = sum(filtered_gt_list, [])

# Drop the pixel_sum column
grouped_gt_codes.drop(columns=['pixel_sum'], inplace=True)
filtered_gt["relabel"] = filtered_gt.index + 1

filtered_gt.drop(columns=['pixel_sum'], inplace=True)

In [6]:
# Set all the labels that are not in the combined_gt_list to 0
cero_labels = merged[~merged['GT'].isin(combined_gt_list)]

In [8]:
# Get the GT list of the cero_labels
cero_labels_list = cero_labels['GT'].tolist()
filtered_gt # Add new row with cero labels
new_row = pd.DataFrame({'4th_tier_ENG': ['cero_labels'], 'GT_list': [cero_labels_list], 'relabel': [0]})

# concat the new row to the filtered_gt
filtered_gt = pd.concat([filtered_gt, new_row], ignore_index=True)



In [9]:
filtered_gt.to_json("filtered_relabel_gt.json", orient="records")

In [107]:
# Add a row that is the cero_label
len(cero_labels_list)

85